In [2]:
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI
import os
import os
from dotenv import load_dotenv
from llama_index.core import (
    VectorStoreIndex, 
)
from llama_index.core.agent.workflow import AgentStream, ToolCallResult
load_dotenv()
from llama_index.core.agent.workflow import ReActAgent
from llama_index.core.workflow import Context
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.vector_stores import MetadataFilter, MetadataFilters
from google import genai
from google.genai import types
from pydantic import BaseModel, Field

from langchain_tavily import TavilySearch

tavily_tool = TavilySearch(max_results=2)


llm_gemini = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-preview-05-20",
    api_key=os.environ["GEMINI_API_KEY"],
)

## Example 1

In [8]:
# Create specialized agents

def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

def web_search(query: str) -> str:
    """Search the web for information."""
    return (
        "Here are the headcounts for each of the FAANG companies in 2024:\n"
        "1. **Facebook (Meta)**: 67,317 employees.\n"
        "2. **Apple**: 164,000 employees.\n"
        "3. **Amazon**: 1,551,000 employees.\n"
        "4. **Netflix**: 14,000 employees.\n"
        "5. **Google (Alphabet)**: 181,269 employees."
    )

math_agent = create_react_agent(
    model=llm_gemini,
    tools=[add, multiply],
    name="math_expert",
    prompt="You are a math expert. Always use one tool at a time."
)

research_agent = create_react_agent(
    model=llm_gemini,
    tools=[web_search],
    name="research_expert",
    prompt="You are a world class researcher with access to web search. Do not do any math."
)

# Create supervisor workflow
workflow = create_supervisor(
    [research_agent, math_agent],
    model=llm_gemini,
    prompt=(
        "You are a team supervisor managing a research expert and a math expert. "
        "For current events, use research_agent. "
        "For math problems, use math_agent."
        "If the user asks a question that requires both, use research_agent first to gather information, then math_agent."
    )
)

config = {"configurable": {"thread_id": "test-thread"}}

# Compile and run
app = workflow.compile()
# result = app.invoke({
#     "messages": [
#         {
#             "role": "user",
#             "content": "what's the combined headcount of the FAANG companies in 2024?"
#         }
#     ]
# })

for step in app.stream(
    {"messages": [("user", "what's the combined headcount of the FAANG companies in 2024?")]},
    config,
    stream_mode="values",
):
    print(step["messages"][-1].pretty_print())
    print("---")

================================ Human Message =================================

what's the combined headcount of the FAANG companies in 2024?
None
---
================================= Tool Message =================================
Name: transfer_to_research_expert

Successfully transferred to research_expert
None
---
================================= Tool Message =================================
Name: transfer_back_to_supervisor

Successfully transferred back to supervisor
None
---
================================= Tool Message =================================
Name: transfer_to_math_expert

Successfully transferred to math_expert
None
---
================================= Tool Message =================================
Name: transfer_back_to_supervisor

Successfully transferred back to supervisor
None
---
================================== Ai Message ==================================
Name: supervisor

The combined headcount of the FAANG companies in 2024 is 1,977,586.
None
---


In [9]:
for step in app.stream(
    {"messages": [("user", "What we were talking about?")]},
    config,
    stream_mode="values",
):
    print(step["messages"][-1].pretty_print())
    print("---")

================================ Human Message =================================

What we were talking about?
None
---
================================== Ai Message ==================================
Name: supervisor

I do not retain past conversations. Please tell me what you would like to discuss.
None
---


## Example 2

In [10]:
import pandas as pd

# dataset con diferentes clientes, su email y el detalle de una compra
df_shoppings = pd.DataFrame({
    "Cliente": ["Juan Pérez", "Ana Gómez", "Luis Martínez", "Ana Gómez", "Carlos López"],
    "Email": ["juan.perez@email.com", "ana.gomez@email.com", "luis.martinez@email.com", "ana.gomez@email.com", "carlos.lopez@email.com"],
    "DetalleCompra": [
        "Compra de laptop - Modelo: Dell XPS 13, Procesador: Intel i7, RAM: 16GB, Almacenamiento: 512GB SSD",
        "Compra de smartphone - Modelo: iPhone 14 Pro, Almacenamiento: 256GB, Color: Negro",
        "Compra de tablet - Modelo: Samsung Galaxy Tab S8, Pantalla: 11 pulgadas, Almacenamiento: 128GB",
        "Compra de audífonos - Modelo: Sony WH-1000XM5, Tipo: Over-ear, Cancelación de ruido activa",
        "Compra de monitor - Modelo: LG UltraGear 27GP850, Tamaño: 27 pulgadas, Resolución: QHD, Frecuencia: 165Hz"
    ]
})

# dataset con diferentes clientes, su email y el detalle de un ticket
df_tickets = pd.DataFrame({
    "Cliente": ["Juan Pérez", "Ana Gómez", "Luis Martínez", "Ana Gómez", "Carlos López"],
    "Email": ["juan.perez@email.com", "ana.gomez@email.com", "luis.martinez@email.com", "ana.gomez@email.com", "carlos.lopez@email.com"],
    "idTicket": [1, 2, 3, 4, 5],
    "DetalleTickets": [
        "Problema con la batería del laptop, no carga correctamente. Modelo: Dell XPS 13, Fecha de compra: 2023-01-15, Estado: En garantía.",
        "Pantalla del smartphone presenta líneas verticales. Modelo: iPhone 14 Pro, Fecha de compra: 2023-02-10, Estado: En garantía.",
        "Tablet no enciende después de la actualización. Modelo: Samsung Galaxy Tab S8, Fecha de compra: 2023-03-05, Estado: Fuera de garantía.",
        "Audífonos no se conectan vía Bluetooth. Modelo: Sony WH-1000XM5, Fecha de compra: 2023-04-20, Estado: En garantía.",
        "Monitor tiene píxeles muertos en la esquina superior derecha. Modelo: LG UltraGear 27GP850, Fecha de compra: 2023-05-30, Estado: En garantía."
    ]
})

In [11]:
from llama_index.core.schema import Document

documents_shoppings = [
    Document(
        text=f"Cliente: {record['Cliente']}, Email: {record['Email']}, Detalle: {record['DetalleCompra']}",
        metadata={"Cliente": record["Cliente"], "Email": record["Email"]},
    )
    for record in df_shoppings.to_dict(orient="records")
]

documents_tickets = [
    Document(
        text=f"Ticket ID: {record['idTicket']}, Cliente: {record['Cliente']}, Email: {record['Email']}, Detalle: {record['DetalleTickets']}",
        metadata={"Cliente": record["Cliente"], "Email": record["Email"], "idTicket": record["idTicket"]},
    )
    for record in df_tickets.to_dict(orient="records")
]

index_shoppings = VectorStoreIndex.from_documents(documents_shoppings, show_progress=True)
index_tickets = VectorStoreIndex.from_documents(documents_tickets, show_progress=True)

/home/rprieto/miniconda3/envs/lang_env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 5/5 [00:00<00:00,  6.36it/s]


In [12]:
class SentimentSchema(BaseModel):
    """Esquema de análisis del sentimiento"""

    sentimiento: str = Field(..., description="Sentimiento del texto")
    score: str = Field(
        ..., description="Puntuación de sentimiento del 0 al 1."
        " Una puntuación más cercana al uno significa un sentimiento más positivo"
        " Una puntuación más cercana al uno significa un sentimiento más negativo"
    )

def analyze_sentiment(text: str) -> dict:
    """Analizar el sentimiento (positivo, negativo, neutro) de un texto dado.
    
    Parameters
    ----------
    text : str
        Texto a analizar.

    Returns
    -------
    dict
        Un diccionario con el sentimiento y la puntuación del texto.
    """

    client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=text,
        config=types.GenerateContentConfig(
            system_instruction="""Eres un experto en análisis de sentimientos. Su tarea consiste en analizar el sentimiento del texto proporcionado.
            Debe seguir el modelo pydantic `SentimentSchema` para devolver la respuesta.""",
            response_mime_type="application/json",
            response_schema=SentimentSchema,
        ),
    )
    return response.text

def insert_ticket(cliente: str, email: str, id_ticket: int, detalle: str) -> str:
    """Inserta un nuevo ticket en el índice de tickets con el detalle del problema reportado.

    Parameters
    ----------
    cliente : str
        Nombre del cliente que reporta el ticket.
    email : str
        Email del cliente que reporta el ticket.
    id_ticket : int
        ID del ticket a insertar.
    detalle : str
        Detalle del problema reportado en el ticket.

    Returns
    -------
    str
        Mensaje de confirmación de inserción.
    """
    try:
        new_ticket = Document(
            text=f"Ticket ID: {id_ticket}, Cliente: {cliente}, Email: {email}, Detalle: {detalle}",
            metadata={"Cliente": cliente, "Email": email, "idTicket": id_ticket},
        )
        index_tickets.insert(new_ticket)
        return "Ticket insertado correctamente."
    except Exception as e:
        return f"Error al insertar el ticket: {str(e)}"

def search_ticket_by_id(id_ticket: int) -> str:
    """Busca un ticket específico por su ID exacto
    y devuelve un resumen detallado del problema reportado.

    Parameters
    ----------
    id_ticket : int
        ID del ticket a buscar.

    Returns
    -------
    str
        Resumen detallado del ticket.
    """
    try:
        id_ticket = int(id_ticket)
        filters = MetadataFilters(filters=[MetadataFilter(key="idTicket", value=id_ticket)])
        engine = index_tickets.as_query_engine(
            filters=filters,
            response_mode="tree_summarize",
            llm=llm_gemini,
            system_prompt="Encuentra y resume el contenido del ticket con este ID exacto."
        )
        return engine.query("Describe detalladamente este ticket").response
    except Exception as e:
        return f"No se encontró el ticket con ID {id_ticket}. Error: {str(e)}"

def search_ticket_by_email(email_client: str) -> str:
    """Busca un ticket específico por el email del cliente
    y devuelve un resumen detallado del problema reportado.

    Parameters
    ----------
    email_client : str
        Email del cliente a buscar.

    Returns
    -------
    str
        Resumen detallado del ticket.
    """
    try:
        filters = MetadataFilters(filters=[MetadataFilter(key="Email", value=email_client)])
        engine = index_tickets.as_query_engine(
            filters=filters,
            response_mode="tree_summarize",
            llm=llm_gemini,
            system_prompt="Encuentra y resume el contenido del ticket correspondiente al cliente con este email."
        )
        return engine.query("Describe detalladamente este ticket").response
    except Exception as e:
        return f"No se encontró el ticket del cliente con email {email_client}. Error: {str(e)}"

def search_shops_by_email(email_client: str) -> str:
    """Busca compras específicas por el email del cliente
    y devuelve un resumen detallado de las compras realizadas.

    Parameters
    ----------
    email_client : str
        Email del cliente a buscar.

    Returns
    -------
    str
        Resumen detallado de las compras.
    """
    try:
        filters = MetadataFilters(filters=[MetadataFilter(key="Email", value=email_client)])
        engine = index_shoppings.as_query_engine(
            filters=filters,
            response_mode="tree_summarize",
            llm=llm_gemini,
            system_prompt="Encuentra y resume el contenido de las compras correspondientes al cliente con este email."
        )
        return engine.query("Describe detalladamente estas compras").response
    except Exception as e:
        return f"No se encontró el ticket del cliente con email {email_client}. Error: {str(e)}"

In [13]:
search_shops_by_email("ana.gomez@email.com")

'No se encontró el ticket del cliente con email ana.gomez@email.com. Error: `llama-index-llms-langchain` package not found, please run `pip install llama-index-llms-langchain`'

In [25]:
sentiment_agent = create_react_agent(
    model=llm_gemini,
    tools=[analyze_sentiment],
    name="sentiment_expert",
    prompt="Usted es un experto en análisis de sentimiento. Analiza el sentimiento del texto proporcionado y " \
 "devuelve una respuesta JSON siguiendo el SentimentSchema."
)

web_search_agent = create_react_agent(
    model=llm_gemini,
    tools=[tavily_tool],
    name="web_search_expert",
    prompt="Eres un experto en búsqueda web. Utilice la herramienta proporcionada para realizar una búsqueda en la web y devolver los resultados."
)

purchases_agent = create_react_agent(
    model=llm_gemini,
    tools=[search_shops_by_email],
    name="purchases_expert",
    prompt="Eres un experto en compras. Utilice la herramienta proporcionada para buscar compras."
)

tickets_agent = create_react_agent(
    model=llm_gemini,
    tools=[search_ticket_by_email, search_ticket_by_id, insert_ticket],
    name="tickets_expert",
    prompt="Eres un experto en tickets. Utilice las herramientas proporcionadas para buscar tickets por correo electrónico o ID, " \
    "o para insertar un nuevo ticket."
)

In [26]:
special_instructions = """
Eres un supervisor de equipo que gestiona a un experto en análisis de sentimientos, un experto en compras y un experto en tickets.
Su deber como agente supervisor es ayudar a los clientes a resolver problemas relacionados con sus compras y tickets.
Debe responder a las preguntas de los clientes utilizando la información disponible sobre los tickets y las compras,
y en el caso de que se reporte un problema, debes insertar un nuevo ticket en el índice de tickets.

En primer lugar, el agente experto en analizar sentimientos debe analizar el sentimiento de los mensajes de los clientes.
Si el sentimiento es negativo, el agente experto en compras debe buscar información sobre las últimas compras del cliente y el agente experto en tickets debe 
insertar un nuevo ticket con el detalle del problema reportado.

Si hay una compra de un producto que corresponde al problema, ES IMPORTANTE que el agente expero en búsqueda web 
haga una búsqueda en la web, a través de la herramienta `tavily_tool`, 
para encontrar y comparar, en primer lugar el precio de mercado del producto, y en segundo lugar el coste de la reparación.

Por último, en base a la pregunta del cliente, debes redactar una respuesta formal tipo email, incluyendo la información relevante de la compra, 
una posible solución al problema planteado y el presupuesto. Si insertaste un ticket debes confirmar al cliente que se ha creado un ticket para su problema.

En caso de que necesites más información para responder a la pregunta, puedes pedir más detalles al usuario.
"""

# Create supervisor workflow
workflow = create_supervisor(
    [sentiment_agent, purchases_agent, tickets_agent],
    model=llm_gemini,
    prompt=special_instructions
)

In [22]:
# Compile and run
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Me ha llegado el movil roto. Tiene la pantalla rota y no se enciende. ana.gomez@email.com"
        }
    ]
})

In [27]:
from pprint import pprint
for s in workflow.compile().stream({"messages": [("user", "Me ha llegado el movil roto. Tiene la pantalla rota y no se enciende. ana.gomez@email.com")]}, stream_mode="values"):
    pprint(s)
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

{'messages': [HumanMessage(content='Me ha llegado el movil roto. Tiene la pantalla rota y no se enciende. ana.gomez@email.com', additional_kwargs={}, response_metadata={}, id='8058bc52-e844-4008-b612-74b5bcf58c06')]}
================================ Human Message =================================

Me ha llegado el movil roto. Tiene la pantalla rota y no se enciende. ana.gomez@email.com
{'messages': [HumanMessage(content='Me ha llegado el movil roto. Tiene la pantalla rota y no se enciende. ana.gomez@email.com', additional_kwargs={}, response_metadata={}, id='8058bc52-e844-4008-b612-74b5bcf58c06'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'transfer_to_sentiment_expert', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'models/gemini-2.5-flash-preview-05-20', 'safety_ratings': []}, name='supervisor', id='run--0191755f-599c-4f36-b467-14e2c9b98f3e-0', tool_cal

In [23]:
print(result['messages'][-1].content)

Lamento mucho lo que ha sucedido con su móvil. Entiendo su frustración.

Dado que no hemos podido encontrar una compra asociada con el email `ana.gomez@email.com`, le pido por favor que nos proporcione más detalles sobre la compra del móvil, como el nombre del producto, la fecha de compra o el número de pedido. Esto nos permitirá buscar el producto y su información para poder ayudarle mejor con la reparación.

Mientras tanto, hemos creado un ticket para su problema.

Para poder avanzar con la resolución de su problema, por favor, envíenos la información de la compra.
